# CSPB-3104 Programming Assignment 9



1) (5 points) Implement Kruskal's algorithm 

Input: An edge list with weights: [(0,1,1), (0,2,2),(1,2,1)]  
Output: A minimum spanning tree in the form of an edge list with weights: [(0, 1, 1), (1, 2, 1)] 

Note: Edge lists are lists of triples (i, j, w), with i < j, which represents an edge between nodes i and j with weight w.  Edges are undirected in this notebook, and you should always return edges in the form (i, j, w), where i < j. Make sure to sort your final edge list in natural order, ie (0, 2, 1) before (1,2,1), (0,1,0) before (0,2,0).

Hint: Look into Python's Set class


In [1]:
def kruskal(edges):
    # Initialize a set for each node
    node_sets = {i: {i} for i in range(max(max(u, v) for u, v, _ in edges) + 1)}
    
    # Function to find the set containing a particular node
    def find_set(node):
        for s in node_sets.values():
            if node in s:
                return s
        return None

    # Sort edges based on their weight
    for i in range(len(edges)):
        for j in range(len(edges) - 1):
            if edges[j][2] > edges[j + 1][2]:
                edges[j], edges[j + 1] = edges[j + 1], edges[j]

    mst = []

    for u, v, w in edges:
        set_u = find_set(u)
        set_v = find_set(v)

        if set_u != set_v:
            # Add the edge to the MST
            mst.append((u, v, w))
            # Merge the two sets
            new_set = set_u.union(set_v)
            for node in new_set:
                node_sets[node] = new_set
    # Final Bubble sort for return
    for i in range(len(mst)):
        for j in range(len(mst) - 1):
            if mst[j][0] > mst[j + 1][0] or (mst[j][0] == mst[j + 1][0] and mst[j][1] > mst[j + 1][1]):
                mst[j], mst[j + 1] = mst[j + 1], mst[j]

    return mst

2. (5 points) Implement Prim's algorithm

Input: An edge list with weights: [(0,1,1), (0,2,2),(1,2,1)]  
Output: A minimum spanning tree in the form of an edge list with weights: [(0, 1, 1), (1, 2, 1)] 

Note: Edge lists are lists of triples (i, j, w), with i < j, which represents an edge between nodes i and j with weight w.  Edges are undirected in this notebook, and you should always return edges in the form (i, j, w), where i < j. Make sure to sort your final edge list in natural order, ie (0, 2, 1) before (1,2,1), (0,1,0) before (0,2,0).

Hint: You can use heapq for the priority queue.

In [2]:
import heapq

def prim(edges):
    graph = {}
    for u, v, w in edges:
        if u not in graph:
            graph[u] = []
        if v not in graph:
            graph[v] = []
        graph[u].append((w, u, v))
        graph[v].append((w, v, u))

    start_node = next(iter(graph))
    mst = []
    visited = set()
    min_heap = []

    def add_edges(node):
        visited.add(node)
        for edge in graph[node]:
            if edge[2] not in visited:
                heapq.heappush(min_heap, edge)

    # Start from the starting node
    add_edges(start_node)

    while min_heap:
        weight, u, v = heapq.heappop(min_heap)
        if v not in visited:
            mst.append((min(u, v), max(u, v), weight))
            add_edges(v)

    # Sort the final MST list before returning
    mst.sort(key=lambda edge: (edge[0], edge[1], edge[2]))
    return mst

3) (15 points)  Finding the most likely mutation tree

You're given a list of bacteria RNA fragments, all from related bacteria which have mutated into separate strains over time.  Your goal is to come up with the most likely sequence of mutations that led to this state of affairs.  

The chance that one bacteria mutated into another depends on the number of differences in their RNA strings. 
The more differences in their RNA strings, the more unlikely it is that the bacteria mutated into each other.  (In fact, exponentially more unlikely -- the probability that k locations changed at the same time is $2^{-k}$).

If we construct a fully connected graph whose nodes represent RNA fragments and each edge has weight $2^{-k}$, where k is the number of differences between RNA strings, then a spanning tree which *maximizes* the *product* of edge weights will be the __most likely mutation tree__.  (Each mututation is assumed to be independent, so the chance that all the mutations in the spanning tree happen is the product of their respective probabilities)

Write a function that takes a list of RNA fragments, constructs an edge list with weights, then returns the most likely mutation tree, along with its probability.  

Note: your algorithm should construct a graph and then run your implementation of Kruskal's algorithm on it.  The difficulty lies in determining the correct graph, so that a minimum sum spanning tree in your graph corresponds to a maximum product spanning tree in the graph described above.

Input: ["adad","adac","acad", "cdac","addd"]  
Output: ([('adad', 'adac', 0.5),
  ('adad', 'acad', 0.5),
  ('adad', 'addd', 0.5),
  ('adac', 'cdac', 0.5)],
 0.0625)

In [3]:
import math

def compute_weight(s1, s2):
    differences = sum(1 for a, b in zip(s1, s2) if a != b)
    return 2 ** -differences

def kruskal_with_sets(edges, num_nodes):
    # Initialize a set for each node
    node_sets = {i: {i} for i in range(num_nodes)}
    
    def find_set(node):
        for s in node_sets.values():
            if node in s:
                return s
        return None

    mst = []
    mst_weight = 0

    for u, v, w in edges:
        set_u = find_set(u)
        set_v = find_set(v)

        if set_u != set_v:
            # Add the edge to the MST
            mst.append((u, v, w))
            mst_weight += w
            # Merge the two sets
            new_set = set_u.union(set_v)
            for node in new_set:
                node_sets[node] = new_set

    return mst, mst_weight

def mutation_tree(rna_fragments):
    num_fragments = len(rna_fragments)
    edges = []
    
    # Construct the graph with weights
    for i in range(num_fragments):
        for j in range(i + 1, num_fragments):
            weight = compute_weight(rna_fragments[i], rna_fragments[j])
            log_weight = -math.log2(weight)  # Use the negative log to convert max product to min sum
            edges.append((i, j, log_weight))
    
    # Sort edges by weight (log_weight)
    for i in range(len(edges)):
        for j in range(len(edges) - 1):
            if edges[j][2] > edges[j + 1][2]:
                edges[j], edges[j + 1] = edges[j + 1], edges[j]

    # Apply Kruskal's algorithm to find the minimum spanning tree
    mst, _ = kruskal_with_sets(edges, num_fragments)
    
    # Convert the MST to use original weights and calculate the probability
    final_mst = []
    probability = 1
    for u, v, log_weight in mst:
        original_weight = 2 ** -log_weight
        final_mst.append((rna_fragments[u], rna_fragments[v], original_weight))
        probability *= original_weight
    
    return final_mst, probability


Testing below

----

In [4]:
## DO NOT EDIT TESTING CODE FOR YOUR ANSWER ABOVE
# Press shift enter to test your code. Ensure that your code has been saved first by pressing shift+enter on the previous cell.
from IPython.core.display import display, HTML
def kruskal_test():
    failed = False
    test_cases = [ 
        ([(0,1,1), (0,2,2),(1,2,1)], [(0, 1, 1), (1, 2, 1)]),
        ([(0,1,2), (0,4,1), (1,2,1), (1,4,2), (2,3,1), (3,4,1)], 
         [(0, 4, 1), (1, 2, 1), (2, 3, 1), (3, 4, 1)]),
        ([(0,1,1), (0,2,2), (0,3,1), (1,4,1), (1,5,2), (2,4,2), 
          (2,6,2), (3,5,2), (3,6,1), (4,7,2), (5,7,2), (6,7,1)], 
          [(0, 1, 1), (0, 2, 2), (0, 3, 1), (1, 4, 1), (1, 5, 2), (3, 6, 1), (6, 7, 1)]),
        ([(0,1,2), (0,2,2), (0,3,1), (1,4,1), (1,5,1), (2,4,2), 
          (2,6,1), (3,5,2), (3,6,2), (4,7,2), (5,7,2), (6,7,1)], 
         [(0, 1, 2), (0, 2, 2), (0, 3, 1), (1, 4, 1), (1, 5, 1), (2, 6, 1), (6, 7, 1)]) 
    ]
    for (test_graph, solution) in test_cases:
        output = kruskal(test_graph)
        if (solution != output):
            s1 = '<font color=\"red\"> Failed - test case: Inputs: graph =' + str(test_graph) + "<br>"
            s2 = '  <b> Expected Output: </b> ' + str(solution) + ' Your code output: ' + str(output)+ "<br>"
            display(HTML(s1+s2))
            failed = True
            
    if failed:
        display(HTML('<font color="red"> One or more tests failed. </font>'))
    else:
        display(HTML('<font color="green"> All tests succeeded! </font>'))
kruskal_test()

/tmp/ipykernel_125/1173355505.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [5]:
## DO NOT EDIT TESTING CODE FOR YOUR ANSWER ABOVE
# Press shift enter to test your code. Ensure that your code has been saved first by pressing shift+enter on the previous cell.
from IPython.core.display import display, HTML
def prim_test():
    failed = False
    test_cases = [ 
        ([(0,1,1), (0,2,2),(1,2,1)], [(0, 1, 1), (1, 2, 1)]),
        ([(0,1,2), (0,4,1), (1,2,1), (1,4,2), (2,3,1), (3,4,1)], 
         [(0, 4, 1), (1, 2, 1), (2, 3, 1), (3, 4, 1)]),
        ([(0,1,1), (0,2,2), (0,3,1), (1,4,1), (1,5,2), (2,4,2), 
          (2,6,2), (3,5,2), (3,6,1), (4,7,2), (5,7,2), (6,7,1)], 
          [(0, 1, 1), (0, 2, 2), (0, 3, 1), (1, 4, 1), (1, 5, 2), (3, 6, 1), (6, 7, 1)]),
        ([(0,1,2), (0,2,2), (0,3,1), (1,4,1), (1,5,1), (2,4,2), 
          (2,6,1), (3,5,2), (3,6,2), (4,7,2), (5,7,2), (6,7,1)], 
         [(0, 1, 2), (0, 2, 2), (0, 3, 1), (1, 4, 1), (1, 5, 1), (2, 6, 1), (6, 7, 1)]) 
    ]
    for (test_graph, solution) in test_cases:
        output = prim(test_graph)
        if (solution != output):
            s1 = '<font color=\"red\"> Failed - test case: Inputs: graph =' + str(test_graph) + "<br>"
            s2 = '  <b> Expected Output: </b> ' + str(solution) + ' Your code output: ' + str(output)+ "<br>"
            display(HTML(s1+s2))
            failed = True
            
    if failed:
        display(HTML('<font color="red"> One or more tests failed. </font>'))
    else:
        display(HTML('<font color="green"> All tests succeeded! </font>'))
prim_test()

/tmp/ipykernel_125/3581563958.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [6]:
## DO NOT EDIT TESTING CODE FOR YOUR ANSWER ABOVE
# Press shift enter to test your code. Ensure that your code has been saved first by pressing shift+enter on the previous cell.
from IPython.core.display import display, HTML
def mutation_test():
    failed = False
    test_cases = [ 
        (["TAT", "CAT", "CAC"],([('TAT', 'CAT', 0.5), ('CAT', 'CAC', 0.5)], 0.25)),
        (["ACATA", "ATCTA", "GTCTA", "GTATA", "GCATA"], 
        ([('ACATA', 'GCATA', 0.5), ('ATCTA', 'GTCTA', 0.5), ('GTCTA', 'GTATA', 0.5), ('GTATA', 'GCATA', 0.5)], 0.0625)),
        (["GATTACA", "CGACTCA", "CATTACA", "CGACATA", "CGTTACA", "CGACACA", "CATTACG", "CGATACA"], 
         ([('GATTACA', 'CATTACA', 0.5), ('CGACTCA', 'CGACACA', 0.5), ('CATTACA', 'CGTTACA', 0.5), 
           ('CATTACA', 'CATTACG', 0.5), ('CGACATA', 'CGACACA', 0.5), ('CGTTACA', 'CGATACA', 0.5), ('CGACACA', 'CGATACA', 0.5)], 0.0078125)),
        (["CATTACA", "GATTACA", "CTTTACA", "CTGGTGA", "CTGTACA", "CTGGTCA", "CTGGTGC", "CTGGACA"], 
        ([('CATTACA', 'GATTACA', 0.5), ('CATTACA', 'CTTTACA', 0.5), ('CTTTACA', 'CTGTACA', 0.5), 
          ('CTGGTGA', 'CTGGTCA', 0.5), ('CTGGTGA', 'CTGGTGC', 0.5), ('CTGTACA', 'CTGGACA', 0.5), ('CTGGTCA', 'CTGGACA', 0.5)], 0.0078125))
    ]
    for (test_graph, solution) in test_cases:
        output = mutation_tree(test_graph)
        if (solution != output):
            s1 = '<font color=\"red\"> Failed - test case: Inputs: graph =' + str(test_graph) + "<br>"
            s2 = '  <b> Expected Output: </b> ' + str(solution) + ' Your code output: ' + str(output)+ "<br>"
            display(HTML(s1+s2))
            failed = True
            
    if failed:
        display(HTML('<font color="red"> One or more tests failed. </font>'))
    else:
        display(HTML('<font color="green"> All tests succeeded! </font>'))
mutation_test()

/tmp/ipykernel_125/2680351308.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
